In [2]:
!conda activate qiskit043
!pip install ipykernel



CondaError: Run 'conda init' before 'conda activate'


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!python -m ipykernel install --user --name=qiskit043 --display-name "Python (qiskit043)"


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Installed kernelspec qiskit043 in /home/jovyan/.local/share/jupyter/kernels/qiskit043


In [1]:
import qiskit
import qiskit_nature

print(qiskit.__version__)
print(qiskit_nature.__version__)


0.24.1
0.6.0


In [2]:
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit import Aer

# Define the H2 molecule
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.735", basis="sto3g")
molecule = driver.run()

# Electronic structure problem
problem = ElectronicStructureProblem(driver)

# Map to qubits
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(problem.second_q_ops()[0])

# Setup VQE with a two-local ansatz
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
optimizer = SLSQP(maxiter=100)
backend = Aer.get_backend("statevector_simulator")

vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=QuantumInstance(backend))
result = vqe.compute_minimum_eigenvalue(qubit_op)

print("H2 Ground State Energy (VQE):", result.eigenvalue.real)


ImportError: cannot import name 'PySCFDriver' from 'qiskit_nature.drivers' (/opt/conda/lib/python3.11/site-packages/qiskit_nature/drivers/__init__.py)

In [3]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.converters import QubitConverter

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit import Aer

# Define the H2 molecule
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.735", basis="sto3g")
molecule = driver.run()

# Electronic structure problem
problem = ElectronicStructureProblem(driver)

# Map to qubits
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(problem.second_q_ops()[0])

# Setup VQE with a two-local ansatz
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
optimizer = SLSQP(maxiter=100)
backend = Aer.get_backend("statevector_simulator")

vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=QuantumInstance(backend))
result = vqe.compute_minimum_eigenvalue(qubit_op)

print("H2 Ground State Energy (VQE):", result.eigenvalue.real)


ModuleNotFoundError: No module named 'qiskit_nature.second_q.converters'

In [7]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import ParityMapper

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit import Aer

# Define the H2 molecule
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.735", basis="sto3g")
molecule = driver.run()

# Build the problem
problem = ElectronicStructureProblem(driver)

# Get the second-quantized Hamiltonian
second_q_ops = problem.second_q_ops()
hamiltonian_op = second_q_ops[0]

# Map to qubits (no QubitConverter in 0.6.0!)
mapper = ParityMapper()
qubit_op = mapper.map(hamiltonian_op)

# Setup VQE
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
optimizer = SLSQP(maxiter=100)
backend = Aer.get_backend("statevector_simulator")

vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=QuantumInstance(backend))
result = vqe.compute_minimum_eigenvalue(qubit_op)

print("H2 Ground State Energy (VQE):", result.eigenvalue.real)


MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."

In [5]:
!pip install pyscf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 116.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [8]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.problems import ElectronicStructureProblem

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator

# Define the H2 molecule
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.735", basis="sto3g")
problem = ElectronicStructureProblem(driver)
second_q_op = problem.second_q_ops()[0]

# Map fermionic operators to qubit operators
mapper = ParityMapper()
qubit_op = mapper.map(second_q_op)

# Variational form (ansatz)
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

# Optimizer
optimizer = SLSQP(maxiter=1000)

# VQE with Estimator primitive
vqe = VQE(Estimator(), ansatz, optimizer)

# Compute ground state energy
result = vqe.compute_minimum_eigenvalue(qubit_op)

print("Ground state energy (H2):", result.eigenvalue.real)


ImportError: cannot import name 'BaseSamplerV2' from 'qiskit.primitives' (/opt/conda/lib/python3.11/site-packages/qiskit/primitives/__init__.py)

In [9]:
!pip install -U "qiskit==0.46.3" "qiskit-algorithms==0.3.0" "qiskit-nature==0.6.0"


  Using cached qiskit_terra-0.46.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached symengine-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached qiskit_terra-0.46.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
Using cached symengine-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (50.4 MB)
  Attempting uninstall: symengine
    Found existing installation: symengine 0.9.2
    Uninstalling symengine-0.9.2:
      Successfully uninstalled symengine-0.9.2
  Attempting uninstall: qiskit-terra━━━━━━━━━━━━ 0/4 [symengine]
    Found existing installation: qiskit-terra 0.24.12m0/4 [symengine]
    Uninstalling qiskit-terra-0.24.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [qiskit-terra]
      Successfully uninstalled qiskit-terra-0.24.1━━━━━━━━━━━━ 1/4 [qiskit-terra]
  Attempting uninstall: qiskit0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [qiskit-terra]
    Found existing installation: qis

In [10]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.problems import ElectronicStructureProblem

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator

# 1. Define molecule (H2 at bond distance 0.735 Å)
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.735", basis="sto3g")
molecule = driver.run()

# 2. Setup problem
problem = ElectronicStructureProblem(molecule)

# 3. Map to qubits
mapper = ParityMapper()
second_q_op = problem.second_q_ops()
hamiltonian = second_q_op[0]
qubit_op = mapper.map(hamiltonian)

# 4. Setup ansatz (trial wavefunction) and optimizer
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz", reps=2)
optimizer = SLSQP(maxiter=200)

# 5. Run VQE
vqe = VQE(Estimator(), ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(qubit_op)

# 6. Print results
print("VQE Ground State Energy (H2):", result.eigenvalue.real, "Ha")

# Classical reference (Hartree-Fock energy)
print("Hartree-Fock Energy:", molecule.hf_energy, "Ha")
print("Nuclear Repulsion Energy:", molecule.nuclear_repulsion_energy, "Ha")


MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."

In [12]:
import sys
!{sys.executable} -m pip install pyscf



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [13]:
from qiskit_nature.second_q.drivers import PySCFDriver

# Try a simple hydrogen molecule
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto3g")
molecule = driver.run()

print("Number of molecular orbitals:", molecule.num_molecular_orbitals)
print("Number of alpha electrons:", molecule.num_alpha)
print("Number of beta electrons:", molecule.num_beta)


MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."

In [14]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.operators import ElectronicIntegrals
from qiskit_nature.units import DistanceUnit
import numpy as np

# Define H2 molecule manually
molecule = MoleculeInfo(
    symbols=["H", "H"],
    coords=[(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)],  # bond length in Angstrom
    charge=0,
    multiplicity=1,
    units=DistanceUnit.ANGSTROM
)

# Create fake integrals (normally generated by a driver like PySCF)
# For H2 in STO-3G, integrals are known and small
one_body = np.array([[-1.252477, -0.475934],
                     [-0.475934, -1.252477]])

two_body = np.zeros((2, 2, 2, 2))
two_body[0,0,0,0] = 0.674493
two_body[1,1,1,1] = 0.674493
two_body[0,1,1,0] = 0.663472
two_body[1,0,0,1] = 0.663472

# Build Hamiltonian
elec_integrals = ElectronicIntegrals.from_raw_integrals(one_body, two_body)
hamiltonian = ElectronicEnergy(elec_integrals)

# Problem object
problem = ElectronicStructureProblem(hamiltonian)

print("Molecule created manually (no PySCF needed).")
print("Number of orbitals:", len(one_body))


Molecule created manually (no PySCF needed).
Number of orbitals: 2


In [18]:
from qiskit.primitives import Estimator
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# 1. Mapper to convert fermionic problem to qubits
mapper = ParityMapper()

# 2. Define ansatz (variational circuit)
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz", reps=2)

# 3. Optimizer
optimizer = SLSQP(maxiter=200)

# 4. Estimator primitive
estimator = Estimator()

# 5. VQE solver (new API requires estimator as 1st argument)
vqe_solver = VQE(estimator, ansatz, optimizer)

# 6. Ground state eigensolver
calc = GroundStateEigensolver(mapper, vqe_solver)

# 7. Run calculation
result = calc.solve(problem)

print("=== H₂ VQE Result ===")
print("Ground state energy:", result.total_energies[0].real, "Hartree")


/opt/conda/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


=== H₂ VQE Result ===
Ground state energy: -2.7878348462914824 Hartree
